<a href="https://colab.research.google.com/github/BrendaCopatti/embedding-classificator/blob/main/classificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISE DOS RESULTADOS

####Imports GDrive

In [1]:
from google.colab import auth
!pip install -U -q PyDrive
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials

####Imports gerais

In [ ]:
import numpy as np
import pandas as pd

####Imports sklearn

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.manifold import TSNE

####Realiza leitura do arquivo de embeddings (1x768)

In [2]:
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
df2 = drive.CreateFile({'id': '1-G1APh0r-oztC4XVBgf8woygRNMpfBKx'})
df2.GetContentFile('embeddings7.csv')
df2 = pd.read_csv('embeddings7.csv', sep = '\t')

In [4]:
df2.head()

,Unnamed: 0,word,embedding,0,1,2,3,4,5,6,...,758,759,760,761,762,763,764,765,766,767
0,0,[CLS],[-1.16593230e+00 -2.52016544e+00 -5.23677349e-...,-1.165932,-2.520165,-0.523677,-0.196077,-1.386048,-0.395073,-0.083243,...,-0.277653,-1.511859,-2.406891,-1.391607,0.006429,0.387169,0.536728,-1.606242,-0.479486,2.991496
1,1,asset,[ 2.88954663e+00 -4.52684784e+00 3.98254871e-...,2.889547,-4.526848,0.398255,-0.456968,4.570643,0.885770,1.231765,...,-1.857725,0.226846,2.933982,0.389052,0.556354,1.637081,-1.516836,-0.468534,-1.305096,-2.487371
2,2,data,[ 1.65825635e-02 3.02072167e+00 2.09216547e+...,0.016583,3.020722,2.092166,1.132092,3.882289,1.284189,2.891476,...,-0.272884,-0.105463,-1.105844,0.383917,-2.484640,-1.634177,2.637579,-2.734812,-3.537330,0.906016
3,3,does,[-3.23484206e+00 -2.61225557e+00 2.57662201e+...,-3.234842,-2.612256,2.576622,-6.168607,1.885877,-4.560091,1.800782,...,-1.774296,-0.832868,-0.484495,3.999647,0.087814,-2.735225,1.936513,-1.262489,0.049269,3.902515
4,4,not,[-9.44261134e-01 -3.55083942e+00 4.54151297e+...,-0.944261,-3.550839,4.541513,-0.458323,0.131612,-4.648763,0.325600,...,-1.356838,2.360378,-0.508321,0.106332,1.937134,0.456787,3.110507,0.049719,2.996107,2.868064


In [5]:
df2.iloc[:, 3:]

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-1.165932,-2.520165,-0.523677,-0.196077,-1.386048,-0.395073,-0.083243,0.056204,-1.239562,-1.029089,...,-0.277653,-1.511859,-2.406891,-1.391607,0.006429,0.387169,0.536728,-1.606242,-0.479486,2.991496
1,2.889547,-4.526848,0.398255,-0.456968,4.570643,0.885770,1.231765,1.497142,1.114057,-3.063864,...,-1.857725,0.226846,2.933982,0.389052,0.556354,1.637081,-1.516836,-0.468534,-1.305096,-2.487371
2,0.016583,3.020722,2.092166,1.132092,3.882289,1.284189,2.891476,1.267866,1.950761,3.014262,...,-0.272884,-0.105463,-1.105844,0.383917,-2.484640,-1.634177,2.637579,-2.734812,-3.537330,0.906016
3,-3.234842,-2.612256,2.576622,-6.168607,1.885877,-4.560091,1.800782,-0.285959,0.612445,2.141300,...,-1.774296,-0.832868,-0.484495,3.999647,0.087814,-2.735225,1.936513,-1.262489,0.049269,3.902515
4,-0.944261,-3.550839,4.541513,-0.458323,0.131612,-4.648763,0.325600,2.853247,2.338148,-0.589544,...,-1.356838,2.360378,-0.508321,0.106332,1.937134,0.456787,3.110507,0.049719,2.996107,2.868064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12056,-0.820400,-4.692802,1.720867,0.631661,-4.051704,-0.925734,1.350187,-2.972397,0.448450,2.583388,...,-1.254266,-2.142297,1.363224,-0.248794,-5.183494,1.145549,-0.457892,-1.989792,-1.203486,-1.226256
12057,3.266142,-5.219947,2.223316,-0.942467,0.928601,-2.086003,-3.642120,-1.378261,-1.517825,3.241320,...,0.317764,0.534057,-2.893503,-1.153612,3.205710,3.676596,0.984629,0.303725,0.266877,0.819345
12058,-1.828222,-4.113334,1.754712,1.343384,4.679567,-3.096134,2.274464,-0.271045,-0.828277,1.595095,...,-0.896423,-4.787981,-0.118785,-1.269734,-4.993939,-0.220703,-1.846077,0.593934,-0.042265,4.414529
12059,-0.555390,-1.136154,-0.729645,0.077940,-0.967427,-5.037321,2.225729,-0.914290,-0.893496,-0.818441,...,2.642987,1.360863,-0.319173,-2.225614,2.521098,0.555902,0.041170,0.332637,-1.450806,1.101676


####Utilizando redutor TSNE

In [6]:
X = np.array(df2.iloc[:, 3:])
X.shape

(12061, 768)

In [7]:
t_sne = TSNE(n_components=2, learning_rate='auto',init='random')
X_embedded= t_sne.fit_transform(X)
X_embedded.shape

(12061, 2)

In [8]:
df = pd.DataFrame(X_embedded, columns=['x', 'y'])

In [9]:
df['word'] = ''

for index, row in df2.iterrows():
    df['word']= df2['word']

In [10]:
df.head()

,x,y,word
0,9.866446,54.241261,[CLS]
1,0.819189,23.220118,asset
2,-19.974607,25.107140,data
3,-74.819405,-13.848521,does
4,-65.277641,25.752764,not


####Realiza leitura do arquivo de classe

In [12]:
df_class = drive.CreateFile({'id': '1ekH65fFYHYYeMRw8K0LjMM9IA73nQIdd'})
df_class.GetContentFile('embedding_xy_class.csv')
df_class = pd.read_csv('embedding_xy_class.csv', sep=',')

In [13]:
df_class['classificador'].count()

1036

In [14]:
df_class = df_class[:1000]

In [15]:
df = df[:1000]

In [16]:
df_class.head()

,Unnamed: 0,word,x,y,classificador
0,0,[CLS],12.043.171,-4.022.194,0.0
1,1,asset,-12.207.879,-21.765.024,0.0
2,2,data,-28.876.194,-13.066.323,0.0
3,3,does,-54.689.423,53.375.206,0.0
4,4,not,-6.650.298,10.505.768,0.0


In [17]:
df['classificador'] = ''

for index, row in df_class.iterrows():
    df['classificador']= df_class['classificador'].astype(int)

####Ajuste do DataFrame para implementar o treino

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   x              1000 non-null   float32
 1   y              1000 non-null   float32
 2   word           999 non-null    object 
 3   classificador  1000 non-null   int64  
dtypes: float32(2), int64(1), object(1)
memory usage: 23.6+ KB


In [19]:
df.iloc[:, 0:2].values

array([[  9.866446 ,  54.24126  ],
       [  0.8191893,  23.220118 ],
       [-19.974607 ,  25.10714  ],
       ...,
       [-34.67996  , -29.43336  ],
       [-49.478603 , -45.80111  ],
       [-21.764597 ,  14.604484 ]], dtype=float32)

In [20]:
X = df.iloc[:, 0:2].values
X

array([[  9.866446 ,  54.24126  ],
       [  0.8191893,  23.220118 ],
       [-19.974607 ,  25.10714  ],
       ...,
       [-34.67996  , -29.43336  ],
       [-49.478603 , -45.80111  ],
       [-21.764597 ,  14.604484 ]], dtype=float32)

In [21]:
y = df['classificador'].values
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

####Separação dos dados de treino e teste

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1, stratify=y)

###Realizando treinamento

---



####Utilizando GridSearch para busca dos melhores parâmetros

In [23]:
param_grid = {
    "criterion" : ['gini', 'entropy'],
    "splitter": ['best', 'random'],
    'min_samples_split': [2, 5, 10],
    "min_samples_leaf": [1, 5, 10]
}

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid)
grid_search.fit(X_train, y_train)

####Verificando scores obtidos

In [25]:
grid_search.cv_results_['mean_test_score']

array([0.7625 , 0.74   , 0.7725 , 0.7875 , 0.78625, 0.81375, 0.77375,
       0.82125, 0.775  , 0.82875, 0.77375, 0.815  , 0.82875, 0.79875,
       0.82875, 0.80875, 0.82875, 0.7975 , 0.76625, 0.7675 , 0.77625,
       0.7825 , 0.77625, 0.8225 , 0.77375, 0.83125, 0.77625, 0.80125,
       0.78   , 0.80375, 0.815  , 0.8075 , 0.815  , 0.82125, 0.81875,
       0.825  ])

####Verificando melho score e melhores parâmetros

In [26]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.83125
{'criterion': 'entropy', 'min_samples_leaf': 5, 'min_samples_split': 2, 'splitter': 'random'}


####Utilizando melhor classificador obtido

In [27]:
new_classifier=grid_search.best_estimator_
y_test_pred=new_classifier.predict(X_test)

####Verificando score

In [28]:
accuracy_score(y_test, y_test_pred)

0.82

In [29]:
precision_recall_fscore_support(y_test, y_test_pred, average='macro')

(0.8050179211469535, 0.7509140228800566, 0.7689641894493646, None)

In [30]:
precision_recall_fscore_support(y_test, y_test_pred, average='micro')

(0.82, 0.82, 0.82, None)

In [31]:
precision_recall_fscore_support(y_test, y_test_pred, average='weighted')

(0.815641577060932, 0.82, 0.8113130535232962, None)